# Introductie Neurale Netwerken met Keras en TensorFlow

- Originele versie: Daniel Moser (UT Southwestern Medical Center)
- Aangepast door Tijmen Muller (tijmen.muller@hu.nl) en Joost Vanstreels (joost.vanstreels@hu.nl) (2021)

Tijdens de cursus Machine Learning gaan we aan de slag met neurale netwerken en convolutionele neurale netwerken (ConvNets), ook wel bekend als deep learning. Tijdens deze workshop gaan we de basisstappen doorlopen om een simpel neuraal netwerk te maken met behulp van de populaire libraries van Google, Keras en TensorFlow.

## Inleiding

### Doel workshop

Het doel van deze workshop is om een beeld te krijgen van de werking van neurale netwerken:
- Wat zijn neurale netwerken?
- Hoe werken ze?
- Voor wat voor een type problemen kun je ze inzetten?
- Wat is het verschil met machine learning technieken zoals decision trees en kNN?
- Wat is de toegevoegde waarde van convolutionele netwerken?

### Opzet workshop

Aan de hand van theoretische uitleg en praktische codevoorbeelden gaan jullie zelfstandig aan de slag met enkele opdrachten:
1. We starten met het opzetten van een simpel neuraal netwerk dat jullie gaan *tweaken* om betere resultaten te krijgen;
2. Daarna gaan jullie het neuraal netwerk uitbreiden met meerdere lagen;
3. Ook dit model gaan jullie tweaken op zoek naar 98% accuracy;
4. Tenslotte gaan we hetzelfde proberen te realiseren met kNN.

### Werkwijze
De werkwijze is als volgt:
- Lees de uitleg goed;
- Voer de code stap-voor-stap uit;
- Voer de opdrachten uit -- de cellen die getagd zijn met `student` moet je zelf invullen;
- Voor het uitvoeren van de opdrachten heb je geen externe bronnen nodig: alle benodigde kennis is gegeven in dit notebook;
- Installeer zelf de benodigde libraries (`keras` en `tensorflow`) en draai het notebook bij voorkeur lokaal. Mocht je het niet werkend krijgen, dan kun je uitwijken naar Google Colab.

## Deel I. Machine learning met neurale netwerken

### Herkennen van handgeschreven cijfers

We gaan aan de slag met de '*Hello World*' van neurale netwerken: het herkennen van de handgeschreven cijfers van de [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Deze dataset bestaat uit een trainset van 60.000 plaatjes met cijfers en een testset van nog eens 10.000 plaatjes.

De plaatjes zien er als volgt uit:

<a title="Josef Steppan, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:MnistExamples.png"><img width="512" alt="MnistExamples" src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png"></a>

### Benodigde libraries 

We hebben een aantal libraries nodig, sommigen kennen jullie al.

In [ ]:
import random  
import time
import platform

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

print("Getest met Python 3.10.9 en TensorFlow 2.10.0")
print("Huidige installatie: Python", platform.python_version(), "en TensorFlow", tf.__version__)
print("Beschikbare GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Quick fix voor onderstaande foutmelding (zie output Jupyter Notebook)
# OMP: Error #15: Initializing libiomp5, but found libiomp5md.dll already initialized.
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### Stap 1. Data exploration 

De MNIST dataset is gebundeld in Keras, we kunnen deze eenvoudig importeren.

In [ ]:
# Inladen van de dataset, deze is al gesplitst in een train- en testset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)
print("x_test shape", x_test.shape)
print("y_test shape", y_test.shape)

**Let op**.
Als je kijkt naar de dataset dan zie je een groot verschil met de datasets die we gezien hebben bij Computational Modelling. Daar hadden we te maken met `.csv`-bestanden met datasets met meerdere kolommen en verschillende datatypes als inhoud. We konden kiezen welke features we als input gingen gebruiken. Met andere woorden: de inhoud van deze datasets was gestructureerd (tabulair). De MNIST dataset bestaat eigenlijk uit 60.000 keer 784 (28 x 28) pixelwaardes... dat is ongestructureerde data. 

Met matplotlib kunnen we een aantal plaatjes uit de dataset bekijken.

In [ ]:
plt.figure(figsize=(9,9))

for i in range(9):
    plt.subplot(3,3,i+1)
    mnist_number = random.randint(0, len(x_train))
    plt.imshow(x_train[mnist_number], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[mnist_number]))
    
plt.tight_layout()

Een plaatje is eigenlijk niks meer dan een twee dimensionaal array met 28 x 28 grijswaardes. Elke pixel is een 8-bit integer met waardes tussen 0 (zwart) en 255 (wit). Hieronder wordt de array getoond van het getal rechtsonder in bovenstaande figuur:

In [ ]:
# just a little function for pretty printing a matrix
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

# now print!        
matprint(x_train[mnist_number])

### Stap 2. Data preparation

#### Features

De input voor het neuraal netwerk is het plaatje. Dat plaatje bestaat eigenlijk uit 784 features: alle 784 pixels. Zo'n 28 x 28 matrix is niet handig als input voor een neuraal netwerk. We moeten er een vector van maken door _flattening_ toe te passen (zie figuur hieronder). We krijgen dan een vector met een lengte van 784 (= 28 x 28).

Daarnaast is het handig om de data te normaliseren. De Z-score gebruiken is hier niet logisch, dus we gaan indexeren waarbij 0 (het minimum) de waarde 0 wordt, 255 (het maximum) wordt 1 en alle tussenliggende waardes worden lineair gemapt naar een waarde tussen 0 en 1.

<img src='https://i.imgur.com/l049B93.png' >

In [ ]:
# Let op: we schakelen hier over van een kleine letter naar een hoofdletter, zodat de originele
# trainingsdata in `x_train` beschikbaar blijft!

X_train = x_train.reshape(60000, 784) # reshape de 60.000 plaatjes van 28 x 28 matrices naar 60.000 784-lengte vectoren.
X_test = x_test.reshape(10000, 784)

X_train = X_train.astype('float32')   # verander datatype van integers naar 32-bit floats
X_test = X_test.astype('float32') 

X_train /= 255                        # normaliseer de pixels door de waarde te delen door de maximale waarde (= 255)
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

#### Target

De target variabele is een 0, 1, 2, ... of 9. Dat zijn de verschillende klasses of categorieën. Kijk bijvoorbeeld maar eens naar de inhoud van een willekeurig item uit `y_train`:

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(X_train[mnist_number].reshape(28,28), cmap='gray', interpolation='none')
print("Dit is een:", y_train[mnist_number])

Bij CM hebben we altijd gewerkt met één targetvariabele die meerdere waardes kon krijgen. Denk aan classificeren waarbij je verschillende klasses kon voorspellen of regressie waarbij je meerdere waardes kon voorspellen.

In dit geval werkt het iets anders: we hebben meerdere targetvariabelen die allemaal corresponderen met één klasse. In dit geval hebben we 10 target variabelen die corresponderen met 0, 1, 2, ... of 9. 

We moeten de target aanpassen naar het volgende formaat:

```
0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]
1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]
2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0]
etc.
```

Het neuraal netwerk gaat straks proberen te voorspellen welk cijfer in een plaatje staat. Net zoals bij decision trees, kNN, etc. zal een voorspelling niet altijd 100% zeker zijn. Het zal vaak een gok zijn met een bepaalde betrouwbaarheid, bijvoorbeeld iets als dit:

```
[0.94, 0, 0, 0, 0, 0, 0.06, 0, .07, 0.11]
```
In dit geval is de voorspelling een '0' met een hoge zekerheid, maar geeft het neurale netwerk ook een kleine kans aan een '6', een '8' en een '9'.

In [ ]:
# One-hot encoding
nb_classes = 10 # aantal klassen

Y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
Y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

print("De", y_train[mnist_number], "is na one-hot encoding:", Y_train[mnist_number])

### Stap 3. Modelling
#### Een eerste supersimpel netwerk
We gaan toewerken naar een fully connected 3-layer network zoals hieronder. Merk op dat je het netwerk van onder naar boven moet lezen: de input (het plaatje van 28 bij 28) staat onderaan, de output (met andere woorden, de voorspelde categorie) staat bovenaan.

<img src="https://i.imgur.com/1MR9U5c.png" />

Maar voordat we dat gaan doen, gaan we eerst een heel simpel netwerk maken dat maar uit één laag bestaat. De input voor het netwerk zijn de 784 pixels en de output zijn de 10 mogelijke klasses:

<img src="https://i.imgur.com/QYNwWcB.png" />

#### Hoe werkt een neuraal netwerk?

Een neuraal netwerk is eigenlijk niks meer dan een heleboel berekeningen. Een netwerk bestaat uit *neuronen* en *verbindingen*. Voor alle 784 pixels zijn neuronen gemaakt die zijn gekoppeld aan alle 10 outputklasses, ook hiervoor zijn neuronen gemaakt. Er zijn in totaal 7.840 verbindingen tussen alle neuronen van de inputlaag en de neuronen van de outputlaag. 

Elke verbinding heeft een gewicht. De waarde van elke pixel is de waarde van de inputneuron, deze wordt vermenigvuldigd met het gewicht van een verbinding. Elk van de 10 outputneuronen heeft als input de som van alle vermenigvuldigingen. De klasse van de outputneuron waarvan de som het hoogste is, is de voorspelling!

#### Model opbouwen
Net zoals bij CM, moet je eerst een model initialiseren. Bij het inladen van de libraries hebben we het `Sequential` model ingeladen. Het sequential model is een lineaire *opstapeling* van lagen en wordt veel gebruikt in de praktijk.

In [ ]:
model = tf.keras.Sequential()

Daarna moeten we handmatig de lagen van het model initialiseren. In dit geval is er maar één laag. Deze code voegt een laag toe aan het model:

```
model.add(tf.keras.layers.Dense(y, input_shape=(x,)))
```

Voor deze laag moet je specificeren hoeveel inputneuronen (`x`) en outputneuronen (`y`) er zijn op basis van de instructies hierboven. Merk op: `(x,)` is geen typo, dat staat voor een vector met lengte `x`.



Neurale netwerken kennen *activatiefuncties*. Dat zijn berekeningen om iets slims te doen met de som van alle vermenigvuldigingen die als input binnen komen. Later zullen we zien waarom dat zo belangrijk is. In dit geval nemen we de _sigmoid function_ als activatiefunctie. Deze zorgt ervoor dat de output nooit lager dan -1 of hoger dan 1 wordt. Dit is een soort van normaliseren. Voor de volledigheid, de formule van de sigmoid is:

$$ a(z) = \frac{1}{1 + {\rm e}^{-z}} $$

<a title="Qef, Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Logistic-curve.svg"><img width="512" alt="Logistic-curve" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/512px-Logistic-curve.svg.png"></a>

In [ ]:
model.add(tf.keras.layers.Activation('sigmoid'))

Dropout zet telkens verschillende willekeurige nodes van de laag op nul. Dit wordt gedaan om _overfitting_ tegen te gaan. Je mag dit weer vergeten op dit moment -- we komen er later in het vak op terug.

In [ ]:
model.add(tf.keras.layers.Dropout(0.2))

Nu hebben we ons model helemaal gebouwd. Hieronder zie je als het goed is de onderstaande samenvatting het eindresultaat:

```
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
=================================================================
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
```

In [ ]:
model.summary()

# We slaan de parameters van het model op, zodat je ze eventueel
# later kunt gebruiken
weights_init = model.get_weights()  

#### Het model *compilen*

Bij CM hebben we gezien dat een model allerlei hyperparameters kent zoals de diepte van een beslisboom, de afstandsfunctie bij _k_-NN of het aantal clusters bij kMeans. Er zijn ook hyperparameters voor het kiezen van een bepaald optimalisatiealgoritme. De belangrijkste reden is dat het optimalisatiealgoritme niet zo belangrijk is bij de modellen die we bij CM gebruikt hebben. Het trainen (of fitten) van het model met het default algoritme lukte altijd goed en snel.

Bij neurale netwerken werkt het trainen van model anders. Het is een stuk complexer waardoor er twee risico's zijn:
- Je vindt geen goede oplossing 
- Het vinden van een goede oplossing duurt lang

Daarom moet je bij een neuraal netwerk goed nadenken over de **loss function** en de **optimizer**. Bij deze cursus zullen we kijken naar trainen met behulp van **gradient descent**. Daarbij is de **learning rate** erg belangrijk.

Gradient descent is het zoeken naar een minimum:
<img src = "https://i.imgur.com/weH0O4U.png" >

Je kunt dat minimum niet zien, dus je moet stapjes in de juiste richting nemen. De grootte van een stap is de learning rate. Kleine stapjes nemen duurt lang, maar te grote stappen is ook niet altijd goed:
<img src = "https://i.imgur.com/jq6rFFa.png" >

Kleine stapjes zijn niet altijd beter. Het risico bij kleine stapjes is dat je blijft hangen bij een lokaal minimum, terwijl er andere (lagere) minima bestaan. Het vinden van het globale minimum is de optimale situatie.

<img src = 'https://i.imgur.com/fft8oPH.png' >

Enfin, hier gaan we later mee aan de slag. Voor nu kiezen we voor de volgende setup van het model:

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Het model trainen

Nu kunnen we eindelijk het model gaan trainen! Zoals gezegd, werkt dat anders dan bij CM. Alle verbindingen moeten berekend worden. Dat doe je stap-voor-stap: 
- Gegeven elk item uit de traindata
- Bepaal de output van het neuraal netwerk
- Vergelijk de output met de echte waarde
- Als de waarde goed is, ga je naar het volgende item
- Als de waarde niet goed is, dan pas je de gewichten van de verbindingen aan die tot het verkeerde antwoord geleid hebben
- Enzovoorts

Dat zijn heel veel berekeningen... en dat kost ook heel veel tijd... daarom ga je bij neurale netwerken niet op zoek naar de optimale oplossing, maar naar een oplossing die goed genoeg is. Of naar de beste oplossing gegeven een bepaalde tijd (_runtime_).

Een manier om te spelen met het trainen, is de keuze voor een **batch size**. Dit bepaalt hoeveel items uit de dataset per stap bekeken worden, voordat de gewichten geüpdate worden. Grotere batch sizes zorgt voor nauwkeuriger trainen, de updates van de parameters zijn dan erg precies. Een kleine batch size (waarbij _online learning_ met batch size gelijk aan 1 de kleinste optie is) zorgt voor snelle updates (na elke batch een update), maar is minder nauwkeurig qua richting. Ook complexiteit speelt een rol bij het kiezen van batch size: een grotere batch gebruikt meer geheugen, maar is computationeel sneller. In de praktijk blijkt een grote batch size de neiging tot overfitting te hebben, maar de oorzaak hiervan is niet helemaal duidelijk. Het zoeken naar de beste batch size is vaak een kwestie van trial en error. Een goed startpunt is een batch size tussen de 32 en de 512 datapunten.

Daarnaast is het aantal **epochs** ook belangrijk. Dit geeft aan hoe vaak je de hele training set wilt gebruiken. Het is niet zo dat je na het bekijken van alle items uit de trainset klaar bent, je kunt de trainset nog een keer gebruiken om het model te verbeteren. En nog een keer. Enzovoorts. Zie het als het studeren voor een tentamen waarbij je een oefententamen vaker maakte om de stof te begrijpen.

In [ ]:
model.fit(X_train, Y_train,
          batch_size=512, epochs=1,
          verbose=1) # verbose zorgt voor een animatie van de voortgang

Je ziet de **loss** en **accuracy**. De accuracy kennen jullie: dat het percentage goed voorspelde cijfers. De loss is het resultaat van de _loss function_ (ook wel _cost function_ genaamd). Het is nog te vroeg om hier dieper op in te gaan, maar onthoud voor nu dat het een foutmaat is, dus we willen hier een zo klein mogelijk getal zien (in de perfecte wereld is de loss gelijk aan 0).

### Stap 4. Evalueren

Leuk, die accuracy op de traindata, maar hoe goed doet ie het op de testdata?! We kijken weer naar de accuracy maar ook naar de score (alhoewel die minder interessant is op dit moment).

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

De algemene evaluatie is belangrijk, maar het is ook goed om specifieke / individuele voorspellingen te bekijken. Hieronder zie je een visualisatie van correct en incorrect voorspelde cijfers:

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
# predicted_classes = model.predict_classes(X_test)  # deprecated
predicted_classes = np.argmax(model.predict(X_test), axis=-1)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
plt.figure(figsize=(9,9))
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(9,9))
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))
    
plt.tight_layout()

### Opdracht 1. De accuracy verhogen

Ga een paar stappen terug en kijk of je de accuracy kunt verhogen. Dat kan op verschillende manieren.
Kijk ook naar de benodigde tijd: duurt het trainen langer of korter?

Speel met:
- epochs
- batch size

**Merk op**: wanneer je meerdere keren achter elkaar `model.fit` aanroept, train je het model steeds verder. Het is dus handig om na elk experiment opnieuw het model op te bouwen om vanaf *scratch* te beginnen. Je kunt ook gebruik maken van de eerder opgeslagen (random) gewichten in `weights_init`, als je dat handig vindt.

Geef hieronder aan welke accuracy je hebt behaald bij welk aantal epochs en welke batch size. Als je in de buurt bent van 92% accuracy op de testset, dan kun je door naar de volgende opdracht.

_Geef hier het antwoord._

### Opdracht 2. Modelling van een completer netwerk
#### Een fully connected 3-layer network
We gaan ons supersimpele netwerk nu uitbouwen tot een fully connected 3-layer network zoals hieronder:

<img src="https://i.imgur.com/1MR9U5c.png" />

#### Waarom meer lagen?!

Het idee is dat een netwerk met meerdere lagen, complexere taken kan uitvoeren. In theorie zou de accuracy voor de MNIST dataset hiermee verhoogd moeten kunnen worden.

### Opdracht 2.a. Het netwerk opbouwen

In [ ]:
# We starten weer met het initialiseren van het model
model = tf.keras.Sequential()

#### De eerste laag

In ons eerste model hadden we een input van 784 en output van 10. Nu is de input weer 784 maar de eerste laag is een  **hidden layer** van 512 neuronen. Voer de volgende zaken uit in cellen hieronder:

1. Gebruik `model.add()` om deze laag toe te voegen aan het model
2. Gebruik de sigmoid als activatiefunctie
3. Voeg de dropout zeroes toe met waarde 0.2

#### De tweede laag

Bij de opvolgende lagen hoef je geen input te definiëren: de vorige laag dient automatisch als input.

- Voeg hieronder nog een laag van 512 neuronen toe aan het model met de sigmoid activatiefunctie en een dropout van 0.2.

####  De output laag

De laatste laag heeft 10 elementen, namelijk de 10 klasses. 

- Voeg een laag van 10 neuronen toe aan het model met de activatiefunctie 'softmax'; een dropout is niet nodig.

In [ ]:
# Laten we even kijken hoe dat eruit ziet
model.summary()

Als het goed is, ziet de samenvatting van het model hierboven er als volgt uit:

```
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
=================================================================
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
```

NB. Met deze twee extra lagen is het totaal aantal te trainen parameters dus bijna een factor 100 gegroeid: 7850 naar __669706__!

#### Compilen

Gebruik dezelfde instellingen voor het compileren van het model.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Trainen

Gebruik dezelfde instellingen voor het trainen van het model.

In [ ]:
model.fit(X_train, Y_train,
          batch_size=512, epochs=1,
          verbose=1)

#### Evalueren

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Opdracht 2.b.

Hoe goed heeft dit *uitgebreidere* netwerk het gedaan ten opzicht van ons 'supersimpele' netwerk? Als het goed is, al veel beter. Ga ook voor dit netwerk spelen met de epochs en batch size en kijk of je de accuracy kunt verhogen. Kijk ook weer naar de benodigde tijd: duurt het trainen langer of korter?

Speel ook met het aantal neuronen in de hidden layers, wat is het effect als je bijvoorbeeld maar 10 neuronen gebruikt in plaats van 512? Voeg eventueel ook hidden layers toe.

**Merk op:** vergeet niet om bij elk experiment het model opnieuw op te bouwen.

Geef hieronder aan welke accuracy je hebt behaald bij welk aantal epochs en welke batch size. Geef ook de structuur van je netwerk aan, dus het aantal layers en het aantal neuronen per layer.  Als je in de buurt bent van 95% accuracy op de testset, dan kun je door naar de volgende opdracht. Als het goed is, kun je nu tot zo'n 98% komen. Als dat gelukt is, kun je verder gaan met de volgende opdracht.

_Geef hier het antwoord._

## Deel II. Blik op de toekomst: deep learning met convolutionele neurale netwerken

### Beperkingen van neurale netwerken

De gevonden resultaten zijn bemoedigend, maar neurale netwerken hebben hun beperkingen. Bekijk onderstaande varianties van een *3* uit de dataset:

<img src="https://i.imgur.com/KfyF1v2.png" />

Het is telkens dezelfde 3 maar dan:
- Verplaatst;
- Verkleurd en ruis op de achtergrond;
- Gedraaid;
- Verkleind;
- Met negatieve kleuren.

In alle gevallen zal ons neuraal netwerk de mist in gaan omdat de pixels van deze drieën afwijken van de pixels van de drieën in de trainset. Je zou dit soort afwijkende drieën kunnen toevoegen aan de trainset maar bij een nieuwe afwijking slaat je netwerk weer eerst de plank mis. 

### Oplossing: kijken zoals mensen kijken

Wanneer wij een plaatje zien, gaan wij niet naar alle pixels kijken en daarmee bepalen of een plaatje een 3 is of iets anders. Wij kijken naar bepaalde patronen, vormen en/of structuren, bijvoorbeeld de drie *pootjes* die een drie heeft of de twee *rondjes* die een acht vormen. We kijken naar scheidingen tussen objecten op een foto of zaken die op de voorgrond of achtergrond staan. Op die manier maken wij onze *voorspellingen*. En dat is precies wat convolutionele neurale netwerken doen.

Voor deze workshop gaat het te ver om ConvNets te behandelen, we bewaren deze materie voor de laatste weken van het vak.

## Deel III. Terug naar het verleden: MNIST met _k_-Nearest Neighbours

Zou het ook mogelijk zijn om cijfers te herkennen met de modellen die we geleerd hebben bij CM? Het is een classificatieprobleem dus regressie en clustering vallen af. Decision Trees zijn niet handig bij grote hoeveelheden features maar kNN kan prima over weg met veel features en in theorie zou je kNN kunnen gebruiken voor de MNIST dataset. Maar werkt het ook in de praktijk?

### Opdracht 3.a. _k_-NN voor MNIST

Duik het verleden in en trek _k_-NN van de plank om een classifier te maken voor de MNIST dataset.

In [ ]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split   
from sklearn.neighbors import KNeighborsClassifier    # kNN from sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Voer de volgende acties uit in de cellen hieronder:

- Gebruik `X_train` en `y_train` om een kNN classifier te fitten. 
- Leg een voorspelling voor `X_test` vast in `y_pred`. 
- Vergelijk deze met `y_test` en bereken de accuracy score.

### Resultaten analyseren

Maak een _confusion matrix_ `cm` door de voorspelde klassen te vergelijken met de werkelijke klassen.

De cel hieronder visualiseert de confusion matrix `cm` voor meer inzicht.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5), dpi=100)

ax = sns.heatmap(pd.DataFrame(cm), annot=True, cmap='Greens', fmt='d')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('voorspelde waarde')
ax.set_ylabel('echte waarde')

plt.show()

### Opdracht 3.b. Accuracy verhogen

Speel met de hyperparameters van kNN om de accuracy te verhogen.
Wat valt je op aan de accuracy van kNN t.o.v. het neuraal netwerk? En de performance van kNN t.o.v. het neuraal netwerk?

Geef hieronder de gevonden resultaten aan. Geef in ieder geval aan hoeveel _neighbors_ je hebt gebruikt, wat de afstandsfunctie was en hoe groot je train en je testset waren. Maak ook een vergelijking in performance tussen het neurale netwerk en _k_-NN.

Als het goed is, kun je nu tot zo'n 97% komen. Als dat gelukt is, ben je klaar.

_Geef hier het antwoord._

### Beperkingen van kNN

kNN doet het harstikke goed voor MNIST. Maar we hebben het hier over plaatjes van 28 x 28 pixels die eigenlijk zwart wit zijn. Wat als we complexere plaatjes zouden gebruiken? Of plaatjes van 1920x1080 pixels? De performance van kNN wordt dan dramatisch en de accuracy is dan ook bedroevend...

#### Fashion MNIST
De Fashion MNIST dataset lijkt op MNIST, maar de plaatjes zijn iets complexer. Kledingstukken lijken qua vorm meer op elkaar en er wordt gebruik gemaakt van meerdere grijstinten. 

<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-Fashion-MNIST-Dataset-1024x768.png" />

De accuracy van kNN duikt voor deze dataset al richting de 85%... niet best dus... en we hebben het hier nog steeds over superkleine plaatjes. Moet je na gaan wat er gebeurt bij grotere plaatjes.

Enfin, als je het leuk vindt om met de Fashion MNIST dataset aan de slag te gaan, gebruik dan de volgende code: 
- `fashion_mnist = tf.keras.datasets.fashion_mnist`
- `(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()`

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
fashion_classes = { 0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}

# Reshape
X_train = x_train.reshape(60000, 784).astype('float32')
X_test = x_test.reshape(10000, 784).astype('float32')

# Normaliseren
X_train /= 255
X_test /= 255

# One-hot encoding
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)

print("Training set met vorm", X_train.shape)
print("Test set met vorm", X_test.shape)

plt.figure(figsize=(2,2))
item = random.randint(0, len(X_train))
plt.imshow(X_train[item].reshape(28,28), cmap='gray', interpolation='none')
print("Dit is een", fashion_classes[y_train[item]], "met label", y_train[item], "=", Y_train[item])

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(512, input_shape=(784,)))
model.add(tf.keras.layers.Activation('sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
weights_init = model.get_weights()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1)
score = model.evaluate(X_test, Y_test)
print(score)

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)  # Gebruik alle processoren

start = time.time()
knn.fit(X_train, y_train)
stop = time.time()

print("k-NN met k =", k, "heeft", train_items, "items gefit in", stop-start, 'seconden')

start = time.time()
y_pred = knn.predict(X_test)
stop = time.time()

knn_score = accuracy_score(y_test_part, y_pred)
print("k-NN heeft", test_items, "items voorspeld in", stop-start, 'seconden met een test accuracy van ', knn_score)

## Conclusies

Hopelijk heb je door deze workshop een beter beeld van wat neurale netwerken zijn en hoe ze werken en hebben we jullie gemotiveerd voor de cursus Machine Learning!